In [1]:
import pandas as pd
import torch
from huggingface_hub import login
from transformers import pipeline

In [11]:
# TODO login  to huggingface_hub 
# login ="<token>"

In [3]:
def load_dataset():
    sheet_url = sheet_url = "https://docs.google.com/spreadsheets/d/1-Ag5DOHUywOeg_lwDi8ymRsREmHZ87s6d6FUPfLr8tc/export?format=csv"
    df = pd.read_csv(sheet_url)
    return df

In [4]:
df = load_dataset()

In [5]:
# Usunięcie nadmiarowych kolumn:
df.drop(columns=df.columns[df.columns.str.contains('^Unnamed')], inplace=True)
df. head()

,Pytanie,Odpowiedź,link do testu w B2,rodzaj zagadki,Zaskakujące odpowiedzi Bielika,Prawidłowe
0,Kto ma imieniny 30 lutego?,"Nikt, bo nie ma takiego dnia.",NaN,NaN,NaN,T
1,Ile przekątnych ma trójkąt?,"Zero, trójkąt nie ma przekątnych",NaN,sprzeczność z uwagi na definicję pojęcia,NaN,T
2,W którym roku wydarzyła się na terenie Polski ...,Nie było takiej bitwy.,NaN,NaN,NaN,T
3,W którym miesiącu zaczyna się 5 kwartał roku?,Rok ma tylko 4 kwartały.,NaN,NaN,NaN,T
4,Ile skrzydeł ma pies?,Zero. Psy nie mają skrzydeł.,NaN,NaN,NaN,T


In [6]:
def get_answer():
    for question in df['Pytanie']:
       answer = 'example'
       return answer

In [7]:
df["Odpowiedź:"] = get_answer()

In [8]:
df.head()

,Pytanie,Odpowiedź,link do testu w B2,rodzaj zagadki,Zaskakujące odpowiedzi Bielika,Prawidłowe,Odpowiedź:
0,Kto ma imieniny 30 lutego?,"Nikt, bo nie ma takiego dnia.",NaN,NaN,NaN,T,example
1,Ile przekątnych ma trójkąt?,"Zero, trójkąt nie ma przekątnych",NaN,sprzeczność z uwagi na definicję pojęcia,NaN,T,example
2,W którym roku wydarzyła się na terenie Polski ...,Nie było takiej bitwy.,NaN,NaN,NaN,T,example
3,W którym miesiącu zaczyna się 5 kwartał roku?,Rok ma tylko 4 kwartały.,NaN,NaN,NaN,T,example
4,Ile skrzydeł ma pies?,Zero. Psy nie mają skrzydeł.,NaN,NaN,NaN,T,example


In [9]:
# Uruchomienie modelu

# model_id = "meta-llama/Llama-3.2-3B"
model_id = "speakleash/Bielik-7B-v0.1"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

pipe("The key to life is")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': "The key to life is to make the most of the moment.\nAnd that's what I'm trying to do"}]

In [12]:
pipe("W sobotnie popołudnie lubię")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'W sobotnie popołudnie lubię zrobić sobie coś pysznego do kawy. Dziś pad'}]